In [1]:
#http://www.ai-start.com/dl2017/html/lesson4-week1.html
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

batch_size = 100

n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    init = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)

#初始化偏执单元对应权值
def bias_variable(shape):
    init = tf.constant(0.1,shape=shape)
    return tf.Variable(init)

#实现卷积层
def conv2d(x,W):
    #input : 输入的要做卷积的图片，要求为一个张量，shape为 [ batch, in_height, in_weight, in_channel ]，其中batch为图片的数量，in_height 为图片高度，in_weight 为图片宽度，in_channel 为图片的通道数，灰度图该值为1，彩色图为3。（也可以用其它值，但是具体含义不是很理解）
    #filter： 卷积核，要求也是一个张量，shape为 [ filter_height, filter_weight, in_channel, out_channels ]，其中 filter_height 为卷积核高度，filter_weight 为卷积核宽度，in_channel 是图像通道数 ，和 input 的 in_channel 要保持一致，out_channel 是卷积核数量。
    #strides： 卷积时在图像每一维的步长，这是一个一维的向量，[ 1, strides, strides, 1]，第一位和最后一位固定必须是1
    #padding： string类型，值为“SAME” 和 “VALID”，表示的是卷积的形式，是否考虑边界。"SAME"是考虑边界，不足的时候用0去填充周围，"VALID"则不考虑
    #use_cudnn_on_gpu： bool类型，是否使用cudnn加速，默认为true
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

#实现池化层
def max_pool_2x2(x):
    #value : 需要池化的输入，一般池化层接在卷积层后面，所以输入通常是feature map，依然是[batch_size, height, width, channels]这样的shape
    #k_size : 池化窗口的大小，取一个四维向量，一般是[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1
    #strides : 窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]
    #padding： 填充的方法，SAME或VALID
    return tf.nn.max_pool(x,[1,2,2,1],strides=[1,2,2,1],padding="SAME")
with tf.name_scope("input"):
    #定义两个输入占位符号
    x = tf.placeholder(tf.float32,[None,784],name="x-input")
    y = tf.placeholder(tf.float32,[None,10],name="y-input")
    with tf.name_scope("x_image"):
        #改变x的格式转化为4D的响亮[batch,in_height,in_width,in_channels]
        x_image = tf.reshape(x,[-1,28,28,1]) #batch：-1 的应用:-1 表示不知道该填什么数字合适的情况下，可以选择，由python通过a和其他的值3推测出来

with tf.name_scope("Conv1"):        
    with tf.name_scope("W_conv1"):
        #初始化的一个卷基层的权值和偏执
        W_conv1 = weight_variable([5,5,1,32]) #5*5的采样窗口，1表示输入通道为1,黑白。32表示输出的通道数--表示使用了32个卷积核（最终会获得32个特征平面）
    with tf.name_scope("b_conv1"):
        b_conv1 = bias_variable([32]) #每一个卷积核都需要一个偏执，所以我们这里要32个偏置值

    with tf.name_scope("relu"):        
        #获取第一层卷积之后的激活值，以及池化层处理以后的隐藏层信息
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
    with tf.name_scope("h_pool1"):
        h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope("Conv2"):        
    with tf.name_scope("W_conv2"):
        #初始化第二个卷积层的权值和偏置
        W_conv2 = weight_variable([5,5,32,64]) #对比上一个的卷积层权值初始化，这里采用5*5采样，32表示输入的维度，64表示使用的卷积核数量（输出的维度）
    with tf.name_scope("b_conv2"):
        b_conv2 = bias_variable([64])

    with tf.name_scope("relu"):
        #二次卷积、池化操作
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
    with tf.name_scope("h_pool2"):
        h_pool2 = max_pool_2x2(h_conv2)

#总结上面两次卷积池化操作
#第一次：卷积后28×28不变，池化后14×14
#第二次：卷积后14×14不变，池化后7×7
#获取了64个7×7二维数据(7*7*64个神经单元)

    with tf.name_scope("h_pool2_flat"):
        #先将第二次池化层处理后的数据扁平化
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])

with tf.name_scope("fc1"):        
    with tf.name_scope("W_fc1"):
        #初始化第一个全链接层权值
        W_fc1 = weight_variable([7*7*64,1024]) #设置第一个全链接层单元数量为1024
    with tf.name_scope("b_fc1"):
        b_fc1 = bias_variable([1024]) #共有1024个目标单元
    with tf.name_scope("relu"):
        #求得第一个全链接层的输出
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
    with tf.name_scope("keep_drop"):
        #设置dropout信息
        keep_drop = tf.placeholder(tf.float32)
    with tf.name_scope("h_fc1_drop"):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_drop)

with tf.name_scope("fc2"):
    with tf.name_scope("W_fc2"):
        #初始化第二个全链接层权值
        W_fc2 = weight_variable([1024,10]) #开始要进行输出了
    with tf.name_scope("b_fc2"):
        b_fc2 = bias_variable([10])
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

with tf.name_scope("loss"):
    #交叉熵处理获取损失
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    tf.summary.scalar("loss",loss)

with tf.name_scope("train_step"):
    #根据损失函数进行优化
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        #返回预测结果到bool列表
        corrent_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope("acc"):
        #返回准确率tf.cast强制转换
        accuracy = tf.reduce_mean(tf.cast(corrent_prediction,tf.float32)) 
        tf.summary.scalar("accuracy",accuracy)
        
merged = tf.summary.merge_all(); #主要就是上面的预测进行了merged操作，所以下面我们直接使用run(merged...)就是进行预测操作

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter("logs/train",sess.graph) #存入两个不同文件
    test_writer = tf.summary.FileWriter("logs/test",sess.graph)
    for iter in range(1001):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_drop:0.5})
        #记录训练集计算的参数
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_drop:1.0})
        train_writer.add_summary(summary,iter)
        #记录测试集计算的参数
        batch_xs,batch_ys = mnist.test.next_batch(batch_size)
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_drop:1.0})
        test_writer.add_summary(summary,iter)
            
        if iter%100 == 0:
            test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_drop:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images[:10000],y:mnist.train.labels[:10000],keep_drop:1.0})
            print("Iter:%d, Test accuracy:%f Train accuracy:%f"%(iter,test_acc,train_acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v